In [ ]:
# !pip install sklearn --upgrade

In [ ]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
import requests
import config
import json
import time
import datetime
import matplotlib.pyplot as plt
from scipy.stats import linregress
import tabletext
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import sys

from pprint import pprint

In [2]:

df = pd.read_csv('data_files/WBNAfullplayerstatslist.csv')
# covid_playoff = pd.read_csv('data_files/player_stats_2019_pst.csv')

In [ ]:
covid_playoff['player_name'] = covid_playoff['First_Name'].str.cat(covid_playoff['Last_Name'], sep =" ")
# covid_playoff = covid_playoff[['player_name', 'Points','Position','game_date']]
covid_playoff

In [3]:
df['player_name'] = df['First_Name'].str.cat(df['Last_Name'], sep =" ")
# covid_playoff = covid_playoff[['player_name', 'Points','Position','game_date']]
df


,Unnamed: 0,First_Name,Last_Name,player_id,Position,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,Assists,...,Team,Home_Away,win,Team_points,Min_played,Crowd,Stadium_Cap,game_id,game_date,player_name
0,0,Camille,Little,9951fcfb-8d31-44e5-ba85-c41ed724763d,F,9,0.0,0.375,20.0,1,...,Sun,1,0,68,29:00:00,5846,9518,e83ca6b1-1257-47d2-9153-ed2c72ad85ab,2015-06-05T23:00:00+00:00,Camille Little
1,1,Jasmine,Thomas,0ac8d024-ec11-4809-83f4-6277c1252617,G,6,100.0,0.250,0.0,4,...,Sun,1,0,68,30:00:00,5846,9518,e83ca6b1-1257-47d2-9153-ed2c72ad85ab,2015-06-05T23:00:00+00:00,Jasmine Thomas
2,2,Kayla,Pedersen,adfc445c-06c0-444a-8856-32cf13a89ea2,F,0,0.0,0.000,0.0,0,...,Sun,1,0,68,10:00,5846,9518,e83ca6b1-1257-47d2-9153-ed2c72ad85ab,2015-06-05T23:00:00+00:00,Kayla Pedersen
3,3,Allison,Hightower,eedb8366-1bff-490e-9dea-5316bbe14559,G,0,0.0,0.000,0.0,0,...,Sun,1,0,68,0,5846,9518,e83ca6b1-1257-47d2-9153-ed2c72ad85ab,2015-06-05T23:00:00+00:00,Allison Hightower
4,4,Elizabeth,Williams,c8672dba-0deb-4cf1-8304-82365865edaa,C-F,5,75.0,0.200,0.0,1,...,Sun,1,0,68,10:00,5846,9518,e83ca6b1-1257-47d2-9153-ed2c72ad85ab,2015-06-05T23:00:00+00:00,Elizabeth Williams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23685,604,JiSu,Park,9a25546d-5f1e-4952-95b7-3e66a76deb92,C,2,0.0,50.000,0.0,0,...,Aces,0,1,98,4:13,13135,18422,d29ed648-b22a-4599-a9d2-b5aff367170c,2019-09-08T20:00:00+00:00,JiSu Park
23686,605,Liz,Cambage,ceee01db-30cb-404f-860c-7acddd358a19,C,21,60.0,64.300,0.0,2,...,Aces,0,1,98,24:04:00,13135,18422,d29ed648-b22a-4599-a9d2-b5aff367170c,2019-09-08T20:00:00+00:00,Liz Cambage
23687,606,Jackie,Young,4fa92d21-9f96-4bab-a9f8-f530a69287ee,G,9,0.0,50.000,100.0,3,...,Aces,0,1,98,16:10,13135,18422,d29ed648-b22a-4599-a9d2-b5aff367170c,2019-09-08T20:00:00+00:00,Jackie Young
23688,607,A'ja,Wilson,6340b055-3c1b-427e-88d6-6f9cdd1fc925,F,20,100.0,100.000,0.0,1,...,Aces,0,1,98,26:41:00,13135,18422,d29ed648-b22a-4599-a9d2-b5aff367170c,2019-09-08T20:00:00+00:00,A'ja Wilson


In [ ]:
covid_dropped = covid_playoff[covid_playoff['Min_played'] != "00:00"]
# df_dropped = df_dropped[df_dropped['Crowd'] != 'Covid']
# df_dropped = df_dropped[df_dropped['Crowd'] != '0']
covid_dropped= covid_dropped[['First_Name', 'Last_Name', 'player_name','player_id', "Points", "Free_Throw_Percent",
                  "Two_Pt_Percent",
                  "Three_Pt_Percent", "Assists",
                  "Rebounds", "Offensive_Rebounds",
                  "Steals", "Personal_Fouls",
                  "Flagrant_Fouls", "Tech_Fouls",
                  "Turnovers",
                  "Home_Away", "win", "Min_played"
                  ]].reset_index(drop = True)
covid_dropped.sort_values(by=["Min_played"], ascending = False)

In [11]:
#dropping Crowd from df_dropped to check status weight v win & try to tweat training % up from 71
# df_dropped = df[df['Min_played'] != "00:00"]
# df_dropped = df_dropped[df_dropped['Crowd'] != 'Covid']
# df_dropped = df_dropped[df_dropped['Crowd'] != '0']
df= df[['First_Name', 'Last_Name', 'player_name','player_id', "Points", "Free_Throw_Percent",
                  "Two_Pt_Percent",
                  "Three_Pt_Percent", "Assists",
                  "Rebounds", "Offensive_Rebounds",
                  "Steals", "Personal_Fouls",
                  "Flagrant_Fouls", "Tech_Fouls",
                  "Turnovers",
                  "Home_Away", "win", "Min_played"
                  ]].reset_index(drop = True)
df

,First_Name,Last_Name,player_name,player_id,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,Assists,Rebounds,Offensive_Rebounds,Steals,Personal_Fouls,Flagrant_Fouls,Tech_Fouls,Turnovers,Home_Away,win,Min_played
0,Camille,Little,Camille Little,9951fcfb-8d31-44e5-ba85-c41ed724763d,9,0.0,0.375,20.0,1,3,2,0,2,0,0,2,1,0,29:00:00
1,Jasmine,Thomas,Jasmine Thomas,0ac8d024-ec11-4809-83f4-6277c1252617,6,100.0,0.250,0.0,4,5,1,0,3,0,0,0,1,0,30:00:00
2,Kayla,Pedersen,Kayla Pedersen,adfc445c-06c0-444a-8856-32cf13a89ea2,0,0.0,0.000,0.0,0,3,1,0,0,0,0,0,1,0,10:00
3,Allison,Hightower,Allison Hightower,eedb8366-1bff-490e-9dea-5316bbe14559,0,0.0,0.000,0.0,0,0,0,0,0,0,0,0,1,0,0
4,Elizabeth,Williams,Elizabeth Williams,c8672dba-0deb-4cf1-8304-82365865edaa,5,75.0,0.200,0.0,1,5,2,0,0,0,0,0,1,0,10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23685,JiSu,Park,JiSu Park,9a25546d-5f1e-4952-95b7-3e66a76deb92,2,0.0,50.000,0.0,0,0,0,0,2,0,0,0,0,1,4:13
23686,Liz,Cambage,Liz Cambage,ceee01db-30cb-404f-860c-7acddd358a19,21,60.0,64.300,0.0,2,9,4,1,3,0,0,2,0,1,24:04:00
23687,Jackie,Young,Jackie Young,4fa92d21-9f96-4bab-a9f8-f530a69287ee,9,0.0,50.000,100.0,3,3,0,0,2,0,0,1,0,1,16:10
23688,A'ja,Wilson,A'ja Wilson,6340b055-3c1b-427e-88d6-6f9cdd1fc925,20,100.0,100.000,0.0,1,0,0,0,5,0,0,3,0,1,26:41:00


In [5]:
df["Min_played"] = df["Min_played"].astype(str)

In [12]:
# time = df['Min_played'].str.split(':', n=2, expand=True)
cols = time.columns
time[cols]= time[cols].apply(pd.to_numeric, errors='coerce')
time.dtypes
time['total_sec_played'] = time[0]*60 + time[1]
time = time['total_sec_played']

df = df.join(time)



In [13]:
time

0        1740.0
1        1800.0
2         600.0
3           NaN
4         600.0
          ...  
23685     253.0
23686    1444.0
23687     970.0
23688    1601.0
23689    1850.0
Name: total_sec_played, Length: 23690, dtype: float64

In [19]:
# df.to_csv('data_files/WBNAfullplayerstatslist.csv')
df.dtypes
df = df[df['total_sec_played'] != 0]
df = df.dropna(subset=['total_sec_played'])
df.to_csv('data_files/WBNAfullplayerstatslist.csv')

df

,First_Name,Last_Name,player_name,player_id,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,Assists,Rebounds,Offensive_Rebounds,Steals,Personal_Fouls,Flagrant_Fouls,Tech_Fouls,Turnovers,Home_Away,win,Min_played,total_sec_played
0,Camille,Little,Camille Little,9951fcfb-8d31-44e5-ba85-c41ed724763d,9,0.0,0.375,20.0,1,3,2,0,2,0,0,2,1,0,29:00:00,1740.0
1,Jasmine,Thomas,Jasmine Thomas,0ac8d024-ec11-4809-83f4-6277c1252617,6,100.0,0.250,0.0,4,5,1,0,3,0,0,0,1,0,30:00:00,1800.0
2,Kayla,Pedersen,Kayla Pedersen,adfc445c-06c0-444a-8856-32cf13a89ea2,0,0.0,0.000,0.0,0,3,1,0,0,0,0,0,1,0,10:00,600.0
4,Elizabeth,Williams,Elizabeth Williams,c8672dba-0deb-4cf1-8304-82365865edaa,5,75.0,0.200,0.0,1,5,2,0,0,0,0,0,1,0,10:00,600.0
5,Kelsey,Bone,Kelsey Bone,93a6bb0d-9500-436e-89b4-33bc8ca6909c,14,0.0,0.583,0.0,2,6,1,2,5,0,0,3,1,0,29:00:00,1740.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23685,JiSu,Park,JiSu Park,9a25546d-5f1e-4952-95b7-3e66a76deb92,2,0.0,50.000,0.0,0,0,0,0,2,0,0,0,0,1,4:13,253.0
23686,Liz,Cambage,Liz Cambage,ceee01db-30cb-404f-860c-7acddd358a19,21,60.0,64.300,0.0,2,9,4,1,3,0,0,2,0,1,24:04:00,1444.0
23687,Jackie,Young,Jackie Young,4fa92d21-9f96-4bab-a9f8-f530a69287ee,9,0.0,50.000,100.0,3,3,0,0,2,0,0,1,0,1,16:10,970.0
23688,A'ja,Wilson,A'ja Wilson,6340b055-3c1b-427e-88d6-6f9cdd1fc925,20,100.0,100.000,0.0,1,0,0,0,5,0,0,3,0,1,26:41:00,1601.0


In [ ]:
covid_dropped["Min_played"] = covid_dropped["Min_played"].astype(str)
time_covid = covid_dropped['Min_played'].str.split(':', n=1, expand=True)
cols_covid = time_covid.columns
time_covid[cols_covid]= time_covid[cols_covid].apply(pd.to_numeric, errors='coerce')
time_covid.dtypes
time_covid['total_sec_played'] = time_covid[0]*60 + time_covid[1]
time_covid = time_covid['total_sec_played']

covid_dropped = covid_dropped.join(time_covid)



In [ ]:
covid_dropped.to_csv('data_files/covid_dropped.csv')
covid_dropped

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler


In [ ]:
lebron_df = df_dropped['player_id']=='0afbe608-940a-4d5d-a1f7-468718c67d91'
lebron_covid = covid_dropped['player_id']=='0afbe608-940a-4d5d-a1f7-468718c67d91'
filter_df = df_dropped[lebron_df]
covid_filter = covid_dropped[lebron_covid]
print(filter_df.shape)
print(covid_filter.shape)
filter_df
covid_filter

In [ ]:
X = filter_df['total_sec_played']
X_covid = covid_filter['total_sec_played']
print(X.shape)
X

In [ ]:
y = filter_df['Points']
y_covid = covid_filter['Points']
print(y.shape)
y.dtype
y


In [ ]:
def  stats(a, b):
    quartilesval = a.quantile([.25, .5, .75])
    lowerq = quartilesval[.25]
    upperq = quartilesval[.75]
    iqr = upperq-lowerq
    lowerbound = lowerq - (iqr*1.5)
    upperbound = upperq + (iqr*1.5)
    upperoutliers = b.loc[a >= upperbound]
    loweroutliers = b.loc[a <= lowerbound]
    outliers=pd.concat([upperoutliers, loweroutliers])
    print(f'The statistical analysis states that there are {len(outliers)} outliers')
    print(f'the upper bound is {upperbound} and the lower bound is {lowerbound}')
    

def linearregress(x,y,a,b):
    (slope, intercept, rvalue, pvalue, stderr)=linregress(x,y)
    regress_values = x * slope + intercept
    line_eq = 'y=' + str(round(slope,2)) +'x+' + str(round(intercept,2))
    plt.scatter(x,y, color='blue')
    plt.plot(x,regress_values,"r-")
    print(f'The r-square is {rvalue}')
    print(line_eq)
    plt.annotate(line_eq,(a,b),fontsize=15, color='red')
    plt.grid()
    
def linearregress2(x,y,a,b):
    (slope, intercept, rvalue, pvalue, stderr)=linregress(x,y)
    regress_values = x * slope + intercept
    line_eq = 'y=' + str(round(slope,2)) +'x+' + str(round(intercept,2))
    plt.scatter(x,y, color='red')
    plt.plot(x,regress_values,"r-")
    print(f'The r-square is for post Covid is {rvalue}')
    print(line_eq)
    plt.annotate(line_eq,(a,b),fontsize=15, color='green')
    plt.grid()

In [ ]:

plt.xlabel('Seconds_Played')
plt.ylabel('Points_Scored')
plt.title('Points vs Time Played')
linearregress(X, y, 1, 1)
stats(filter_df['Points'], filter_df)
linearregress2(X_covid, y_covid, 25, 25)
stats(covid_filter['Points'], covid_filter)